In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

/home/ubuntu/sustaingym


# Scripts for generating events and running environment

In [2]:
from datetime import datetime, timedelta
import os
import pytz

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")

from sustaingym.data.load_moer import load_monthly_moer
from sustaingym.envs.evcharging import EVChargingEnv, RealTraceGenerator, GMMsTraceGenerator
from sustaingym.algorithms.evcharging.base_algorithm import GreedyAlgorithm, RandomAlgorithm, RLAlgorithm, MPC


test_ranges = (
    ('2019-05-01', '2019-08-31'),
    ('2019-09-01', '2019-12-31'),
    ('2020-02-01', '2020-05-31'),
    ('2021-05-01', '2021-08-31'),
)


## Real trace generator

In [3]:
for site in ['caltech', 'jpl']:
    for test_range in test_ranges:
        print('testing: ', site, test_range)
        rtg = RealTraceGenerator(site, test_range)
        for _ in range(123):  # 4 months -> 123 days maximum
            _, _, num_plug_events = rtg.get_event_queue()
        print(rtg)

testing:  caltech ('2019-05-01', '2019-08-31')
RealTracesGenerator from the Caltech site from 2019-05-01 to 2019-08-31. Current day 2019-08-31. 
testing:  caltech ('2019-09-01', '2019-12-31')
RealTracesGenerator from the Caltech site from 2019-09-01 to 2019-12-31. Current day 2019-09-01. 
testing:  caltech ('2020-02-01', '2020-05-31')
RealTracesGenerator from the Caltech site from 2020-02-01 to 2020-05-31. Current day 2020-02-02. 
testing:  caltech ('2021-05-01', '2021-08-31')
RealTracesGenerator from the Caltech site from 2021-05-01 to 2021-08-31. Current day 2021-08-31. 
testing:  jpl ('2019-05-01', '2019-08-31')
RealTracesGenerator from the Jpl site from 2019-05-01 to 2019-08-31. Current day 2019-08-31. 
testing:  jpl ('2019-09-01', '2019-12-31')
RealTracesGenerator from the Jpl site from 2019-09-01 to 2019-12-31. Current day 2019-09-01. 
testing:  jpl ('2020-02-01', '2020-05-31')
RealTracesGenerator from the Jpl site from 2020-02-01 to 2020-05-31. Current day 2020-02-02. 
testing: 

## GMMs trace generator

In [4]:
for site in ['caltech', 'jpl']:
    for test_range in test_ranges:
        print('testing: ', site, test_range)
        gmmg = GMMsTraceGenerator(site, test_range)
        for _ in range(123):  # 4 months -> 123 days maximum
            _, _, num_plug_events = gmmg.get_event_queue()
        print(gmmg)

testing:  caltech ('2019-05-01', '2019-08-31')
GMMsTracesGenerator from the Caltech site from 2019-05-01 to 2019-08-31. Sampler is GMM with 30 components. 
testing:  caltech ('2019-09-01', '2019-12-31')
GMMsTracesGenerator from the Caltech site from 2019-09-01 to 2019-12-31. Sampler is GMM with 30 components. 
testing:  caltech ('2020-02-01', '2020-05-31')
GMMsTracesGenerator from the Caltech site from 2020-02-01 to 2020-05-31. Sampler is GMM with 30 components. 
testing:  caltech ('2021-05-01', '2021-08-31')
GMMsTracesGenerator from the Caltech site from 2021-05-01 to 2021-08-31. Sampler is GMM with 30 components. 
testing:  jpl ('2019-05-01', '2019-08-31')
GMMsTracesGenerator from the Jpl site from 2019-05-01 to 2019-08-31. Sampler is GMM with 30 components. 
testing:  jpl ('2019-09-01', '2019-12-31')
GMMsTracesGenerator from the Jpl site from 2019-09-01 to 2019-12-31. Sampler is GMM with 30 components. 
testing:  jpl ('2020-02-01', '2020-05-31')
GMMsTracesGenerator from the Jpl site

## Rewards from policies

In [5]:
policies = ['random', 'full', 'none', 'selective_full']

for policy in policies:
    rewards = []
    reward_comps = {'profit': 0, 'carbon_cost': 0, 'excess_charge': 0}
    gmmg = GMMsTraceGenerator('caltech', test_ranges[0])
    env = EVChargingEnv(gmmg, action_type='discrete', moer_forecast_steps=36)

    num_episodes = 5
    for _ in range(num_episodes):
        obs = env.reset()
        done = False
        tot_reward = 0
        while not done:
            if policy == 'random':
                action = np.random.randint(0, 5, size=(54,))
            elif policy == 'full':
                action = np.full((54,), 4)
            elif policy == 'none':
                action = np.zeros((54,))
            else:
                action = np.where(obs['demands'] > 0, 4, 0)
            obs, reward, done, info = env.step(action)

            tot_reward += reward
            for reward_comp in info['reward']:
                reward_comps[reward_comp] += info['reward'][reward_comp]
        rewards.append(tot_reward)
    print(f'{policy} rewards: ', rewards)  # rewards per num_episodes
    print('reward components:', reward_comps)  # total reward contribution over num_episodes


random rewards:  [-9.20538625756031, -16.095760394232656, -40.353458203385635, -32.150978309804614, -19.847337928997227]
reward components: {'profit': 39.14599951563122, 'carbon_cost': 144.0161539449463, 'excess_charge': 12.782766664665491}
full rewards:  [-88.93214742348485, -76.00004425021787, -64.92499589865847, -41.43152301273302, -80.06867687006691]
reward components: {'profit': 37.86346411370879, 'carbon_cost': 316.83008619075076, 'excess_charge': 72.39076537812032}
none rewards:  [0.0, 0.0, 0.0, 0.0, 0.0]
reward components: {'profit': 0.0, 'carbon_cost': 0.0, 'excess_charge': 0.0}
selective_full rewards:  [10.726320747729439, 2.7434102707532584, 1.3634530237333227, 1.9140559962278771, 10.532486064969884]
reward components: {'profit': 44.10525285538047, 'carbon_cost': 16.648518143200974, 'excess_charge': 0.1770086087657736}


## Check seed setting is done correctly

In [3]:
print('RealTraceGenerator')
for sequential in [False, True]:
    for random_seed in [None, 11]:
        print(f'sequential {sequential}, random_seed {random_seed}')
        for _ in range(3):
            gen = RealTraceGenerator('caltech', test_ranges[1], sequential=sequential, random_seed=random_seed)

            num_events = []
            for _ in range(7):
                _, _, num_event = gen.get_event_queue()
                num_events.append(num_event)
            print(num_events)

print('GMMsTraceGenerator')
for random_seed in [None, 11]:
    print(f'random_seed {random_seed}')
    for _ in range(3):
        gen = GMMsTraceGenerator('caltech', test_ranges[1], random_seed=random_seed)
        num_events = []
        for _ in range(7):
            _, _, num_event = gen.get_event_queue()
            num_events.append(num_event)
        print(num_events)

RealTraceGenerator
sequential False, random_seed None
[29, 10, 36, 26, 23, 27, 31]
[34, 32, 35, 34, 28, 28, 4]
[31, 31, 9, 4, 1, 6, 30]
sequential False, random_seed 11
[32, 6, 20, 33, 23, 33, 30]
[32, 6, 20, 33, 23, 33, 30]
[32, 6, 20, 33, 23, 33, 30]
sequential True, random_seed None
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
sequential True, random_seed 11
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
[5, 11, 31, 30, 35, 36, 10]
GMMsTraceGenerator
random_seed None
[36, 9, 6, 29, 7, 30, 38]
[30, 31, 31, 35, 4, 11, 33]
[7, 33, 1, 8, 6, 31, 7]
random_seed 11
[35, 10, 39, 38, 30, 19, 9]
[35, 10, 39, 38, 30, 19, 9]
[35, 10, 39, 38, 30, 19, 9]


## Timestep Greedy and MPC

### Discrete Case

In [4]:
import pdb

ga = GreedyAlgorithm(project_action=True)
mpc1 = MPC(lookahead=1)
mpc2 = MPC(lookahead=2)
mpc6 = MPC(lookahead=6)
mpc12 = MPC(lookahead=12)
mpc36 = MPC(lookahead=36)

lbls = ['mpc1']
algs = [mpc1]
# lbls = ['greedy', 'mpc1', 'mpc2', 'mpc6', 'mpc12', 'mpc36']
# algs = [ga, mpc1, mpc2, mpc6, mpc12, mpc36]

DEFAULT_DATE_RANGES = (
    ('2019-05-01', '2019-08-31'),
    ('2019-09-01', '2019-12-31'),
    ('2020-02-01', '2020-05-31'),
    ('2021-05-01', '2021-08-31'),
)

# bug fix: commit 18be9933cbf14e19e17332c9a870f480471eea86
# 2019-08-14: StationOccupiedError: Station CA-303 is occupied with ev 2_39_139_28_2019-08-15 07:07:28.618042.
# 2019-10-25: StationOccupiedError: Station CA-317 is occupied with ev 2_39_91_437_2019-10-26 07:36:37.638121. -> change mask: mask = (self.day.day == max_depart.dt.day) instead of mask = (df['arrival'].dt.day == max_depart.dt.day)
# 2020-03-15: StationOccupiedError: Station CA-303 is occupied with ev 2_39_139_28_2020-03-16 07:51:17.415039.

DATE_FORMAT = '%Y-%m-%d'
def num_days_in_period(xs) -> int:
    """Returns the number of days in period."""
    dts = tuple(datetime.strptime(x, DATE_FORMAT) for x in xs)
    td = dts[1] - dts[0]
    return td.days + 1

for alg, lbl in zip(algs, lbls):
    for season in DEFAULT_DATE_RANGES:
        gen = RealTraceGenerator('caltech', season, sequential=True)
        env = EVChargingEnv(gen)
        rewards, breakdown = alg.run(num_days_in_period(season), env)

        print(f'{lbl} average reward: ', np.mean(rewards))
        print(f'{lbl} rewards: ', rewards)
        print(f'{lbl} reward breakdown: ', breakdown)

100%|██████████| 123/123 [08:28<00:00,  4.13s/it]


mpc1 average reward:  7.410445745394435
mpc1 rewards:  [17.611852001760465, 11.448670037614956, 7.985192220906874, 2.532929820790012, 4.176859627738011, 12.365029189409828, 14.17219046398295, 11.264549520767648, 10.411906492757872, 10.561962344299584, 3.5108945420441184, 3.02229013132516, 16.97669532028571, 11.332930287640055, 11.220862345782631, 12.074439985514118, 14.354725924204056, 3.3824462410599696, 4.525322426872741, 15.721941822425856, 15.589723767711682, 10.710548717430974, 9.325908827321927, 10.041486342215087, 1.5577960668462778, 1.8944209934543916, 3.3740567292451296, 11.393019469259556, 12.041897462487684, 11.9524210946233, 10.337496287665852, 2.156338751693035, 2.552634520257798, 14.155571434432602, 11.589002950875702, 9.953536333828946, 9.83566177896273, 13.401318649880784, 2.36242955768457, 2.1580347025392252, 7.87894504608418, 5.462143927027356, 10.69286232039029, 9.457893157074517, 14.594033671808278, 6.082983311262314, 1.8734046584165327, 8.50700680444646, 10.0031633

100%|██████████| 122/122 [08:55<00:00,  4.39s/it]


mpc1 average reward:  6.952235601416942
mpc1 rewards:  [0.7894010243878143, 2.7265030346381063, 6.61610753573998, 7.665468052922636, 7.827165146129333, 8.04432969120473, 2.4744573238958645, 2.1771152052274836, 14.649042464637708, 8.249903579071134, 9.671667994354133, 9.602741274276582, 7.353805117861956, 3.41740651046765, 1.6172736654832343, 8.487804753640399, 4.590118507453238, 9.050134211639602, 11.550236588056169, 10.4133473417773, 0.9334230032940265, 2.1036614809385576, 13.10871392252195, 7.857973669218443, 8.451251392104956, 10.033600059684902, 10.814211443851745, 0.994100578605656, 2.9008364458881544, 11.40326896461515, 13.395796456187824, 8.952760804200834, 10.893795709596636, 12.534941293593596, 1.7724280456082808, 0.13559649001477792, 11.35602597874921, 11.543417384982018, 9.721650580925044, 10.39003925690486, 8.689221439687659, 3.9857267224679385, 1.2794089121843815, 12.089453484347226, 8.882090228700484, 8.234318249701051, 10.246700925662692, 11.009921924959864, 2.0914770207

100%|██████████| 121/121 [10:19<00:00,  5.12s/it]


mpc1 average reward:  2.821764269448531
mpc1 rewards:  [1.3757655969519373, 2.086546778261297, 10.834143115049326, 11.91075207819866, 14.769309264478313, 10.029833901492848, 12.626465400960143, 2.6132799119339456, 2.7587939386981812, 9.377536626967977, 15.019545302161271, 14.548587543858376, 13.859865476552129, 9.19117512339146, 0.9168226381166227, 1.1510554994240527, 2.5307532702451403, 11.742599686403407, 12.59595948015232, 13.23194005225372, 9.558470626451426, 2.416420534214556, 0.6151990165220479, 8.061383205815305, 9.915439857378988, 10.205381702630099, 10.269805130171445, 9.177516784053216, 3.5513181464955026, 1.123250317702272, 12.53160660073345, 9.485939410681167, 10.169572519874404, 8.61470239587391, 9.972882381651944, 0.9242112779725825, 2.3185855177898036, 9.78961632219427, 6.694243029731545, 8.075630302642615, 4.087244066165285, 2.492654596589272, 0.22001049603635514, 2.25196400913677, 0.861356092089856, 1.4008683880887889, 0.8246101985667648, 0.6996959315997833, 0.0, 0.0, 

100%|██████████| 123/123 [08:42<00:00,  4.25s/it]

mpc1 average reward:  2.186709295304183
mpc1 rewards:  [0.3868741283088463, 0.0, 0.8689697471416109, 3.038588669577814, 2.006021629877654, 1.0813692481363455, 2.786496148885035, 0.4659200000000028, 0.7715734650263332, 1.5080532804422504, 1.639709133606255, 2.8475444091252267, 1.1349194981710715, 2.6781508091157114, 1.0686253237180243, 1.4537429034369589, 1.0146109999317325, 1.8156312243966373, 3.0949829409447496, 1.2548008896697063, 2.0326211361193827, 0.7847458266260414, 0.6742085823639093, 3.4808683581918896, 3.4719871857525386, 1.7973451532230305, 1.2614973425257958, 1.8116667685353471, 0.7589198109571695, 0.4171613755729792, 0.3388366579104569, 1.3448581583718273, 3.426047060715975, 1.7840879438824027, 1.2856618718039485, 1.5035033597699154, 1.9585412392174784, 1.1670225591484862, 0.8035723083879619, 2.423774150243565, 1.8999790326937698, 2.783678967776185, 0.9634725203668606, 0.4734068489513869, 2.649339976113318, 2.177681065276564, 1.3960537734980385, 1.863511198908633, 2.6757977

### Continuous Case

In [6]:
for alg, lbl in zip(algs, lbls):
    gen = RealTraceGenerator('caltech', ('2019-06-03', '2019-06-09'), sequential=True)
    env = EVChargingEnv(gen, action_type='continuous')
    rewards, breakdown = alg.run(7, env)

    print(f'{lbl} average reward: ', np.mean(rewards))
    print(f'{lbl} rewards: ', rewards)
    print(f'{lbl} reward breakdown: ', breakdown)

Simulating days of charging


100%|██████████| 7/7 [01:08<00:00,  9.84s/it]


greedy average reward:  9.15312305135341
greedy rewards:  [14.73946295170452, 11.580331177092566, 9.988362172844129, 9.843150197362618, 13.403252969331234, 2.3592671885995906, 2.1580347025392252]
greedy reward breakdown:  {'profit': 73.72686496579581, 'carbon_cost': 9.653668939655637, 'excess_charge': 0.0013346666666666687}
Simulating days of charging


100%|██████████| 7/7 [00:30<00:00,  4.41s/it]


mpc1 average reward:  9.074372292724918
mpc1 rewards:  [14.221004058892053, 11.582590121781422, 9.961209517105905, 9.83231499383925, 13.406185466316995, 2.3592671885995906, 2.1580347025392252]
mpc1 reward breakdown:  {'profit': 73.10548875144511, 'carbon_cost': 9.584744035704347, 'excess_charge': 0.0001386666666666669}
Simulating days of charging


...

mpc6 average reward:  9.132565649341212
mpc6 rewards:  [14.375190524270714, 11.671187575681968, 10.098768997422443, 9.828358337244667, 13.427159174181057, 2.3716885473568032, 2.1556063892308193]
mpc6 reward breakdown:  {'profit': 72.89555446728872, 'carbon_cost': 8.96667846007515, 'excess_charge': 0.0009164618255215675}
Simulating days of charging


100%|██████████| 7/7 [00:49<00:00,  7.02s/it]


mpc12 average reward:  9.079287577546324
mpc12 rewards:  [14.33771424234256, 11.671406267863418, 9.918192930069637, 9.690321309337019, 13.421648793684708, 2.379744160659251, 2.1359853388676804]
mpc12 reward breakdown:  {'profit': 71.97790507862565, 'carbon_cost': 8.420916129664683, 'excess_charge': 0.001975906137184119}
Simulating days of charging


100%|██████████| 7/7 [01:28<00:00, 12.64s/it]

mpc36 average reward:  8.792959617789156
mpc36 rewards:  [14.386969259443795, 11.296497433271572, 8.763924318459628, 9.530409927916946, 13.332347607533718, 2.263419242305014, 1.9771495355934363]
mpc36 reward breakdown:  {'profit': 68.60606645319396, 'carbon_cost': 7.050200858590292, 'excess_charge': 0.005148270079856753}
